It is highly recommended to use a powerful **GPU**, you can use it for free uploading this notebook to [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb).
<table align="center">
 <td align="center"><a target="_blank" href="https://colab.research.google.com/github/ezponda/intro_deep_learning/blob/main/class/CNN/Object_Tracking_Counting.ipynb">
        <img src="https://colab.research.google.com/img/colab_favicon_256px.png"  width="50" height="50" style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/ezponda/intro_deep_learning/blob/main/class/CNN/Object_Tracking_Counting.ipynb">
        <img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png"  width="50" height="50" style="padding-bottom:5px;" />View Source on GitHub</a></td>
</table>

# Object Tracking with Ultralytics YOLO

Object tracking extends the capabilities of object detection by not only identifying and localizing objects within each frame of a video but also maintaining a unique identifier for each detected object across the frames. This process is crucial for applications that require monitoring the movement and behavior of objects over time, such as surveillance, sports analytics, and advanced driver-assistance systems (ADAS).

## Differences Between Object Tracking and Detection

- **Object Detection** is the process of identifying objects within a single image frame and classifying them into predefined categories. It provides the location of the object typically in the form of bounding boxes.

- **Object Tracking**, on the other hand, follows the identified objects from one frame to the next. This involves detecting the object in the initial frame and then tracking the object’s movement across subsequent frames, maintaining a consistent identifier for each object.

## Features of Ultralytics YOLO Object Tracking

Ultralytics YOLO offers enhanced object tracking capabilities:

- **Real-Time Tracking**: Capable of tracking objects seamlessly in high-frame-rate videos, making it suitable for real-time applications.
- **Multiple Tracker Support**: Provides options to select from various established tracking algorithms.
- **Customizable Tracker Configurations**: Allows adjustment of tracker parameters to suit specific requirements, enhancing flexibility and performance.

## Available Trackers in Ultralytics YOLO

Ultralytics YOLO supports different tracking algorithms, including:

- **BoT-SORT**: Default tracking algorithm, suitable for a variety of tracking needs. Enable with `tracker='botsort.yaml'`.
- **ByteTrack**: Known for its high performance, especially in dense scenes. Enable with `tracker='bytetrack.yaml'`.

## Implementing Object Tracking

To apply object tracking with Ultralytics YOLO, you can use the following code snippets:

### Loading and Running a Tracker

```python
from ultralytics import YOLO

# Load a pretrained model
model = YOLO('yolov8n.pt')  # Substitute with yolov8n-seg.pt or yolov8n-pose.pt as needed

# Perform tracking on a video source
results = model.track(source="path/to/video.mp4", show=True, tracker="botsort.yaml")  # Use "bytetrack.yaml" for ByteTrack
```

This code will run the specified tracker on the video source, displaying the tracked objects in real-time.

### Configuration and Customization

Tracking configuration can be customized similar to prediction settings, including confidence and IoU thresholds:

```python
# Configure the tracking parameters
results = model.track(source="path/to/video.mp4", conf=0.3, iou=0.5, show=True)
```

You can also use a custom tracker configuration by modifying a YAML file from the `ultralytics/cfg/trackers` directory:


In [ ]:
#!pip install torch torchvision torchaudio
#!pip install ultralytics
#!pip install opencv-python

In [ ]:
import os
# Set the environment variable within the notebook
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

### Optional: Web cam Local (Not for Colab) 

In [ ]:
from PIL import Image
import time
from IPython.display import Image as IPyImage

cap = cv2.VideoCapture(0)
time.sleep(1)  ### letting the camera autofocus

axes = None
NUM_FRAMES = 100  # you can change this
processed_imgs = []
for i in range(NUM_FRAMES):
    # Load frame from the camera
    ret, frame = cap.read()
    
    # Run the model
    results = model.track(frame, persist=True, verbose=False)
    result = results[0]
    
    annotated_image_bgr = result.plot()
    annotated_image_rgb = annotated_image_bgr[..., ::-1]  # Convert BGR to RGB
    
    img = Image.fromarray(np.uint8(annotated_image_rgb))
    processed_imgs.append(img)
    cv2.imshow("test", annotated_image_bgr)
    cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()

In [ ]:
## create gif
processed_imgs[0].save('web_cam_tracking.gif',
                       format='GIF',
                       append_images=processed_imgs[1:],
                       save_all=True,
                       duration=100,
                       loop=0)

In [ ]:
#IPyImage('web_cam_tracking.gif', format='png', width=15 * 40, height=3 * 40) 

### Traffic scene

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
def show_frame(img):
    """
    Displays an image frame. Automatically detects the execution environment.
    Args:
        img (numpy.ndarray): The image frame to be displayed.
    """
    try:
        from google.colab.patches import cv2_imshow
        from IPython.display import clear_output
        clear_output(wait=True)  # Clear the output to prevent image accumulation
        cv2_imshow(img)  
    except ImportError:
        cv2.imshow("YOLOv8 Tracking", img)
        
traffic_url_1 = "https://github.com/ezponda/intro_deep_learning/raw/main/images/road_traffic_video_for_object_recognition_short.mp4"


# Open the video file
cap = cv2.VideoCapture(traffic_url_1)

# Loop through the video frames
while cap.isOpened():
    success, frame = cap.read()
    
    if success:
        # Run YOLOv8 tracking on the frame
        results = model.track(frame, persist=True, verbose=False)
        result = results[0]

        # Visualize and display the results
        annotated_frame = result.plot()
        show_frame(annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): # Press 'q' to quit
            break
    else:
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(result)

In [ ]:
help(result)

In [ ]:
help(result.boxes)

In [ ]:
print(result.boxes)

In [ ]:
print('Classes detected', result.boxes.cls.numpy())
print('Total classes', result.names)

## Question 1: Counting the Total Appearances of Trucks

Count the total number of truck appearances across all frames in the video. Trucks are identified with the class index that appears in `result.names` . Your task is to complete the code such that it correctly counts the number of times trucks appear in the video frames. Take note of how the results are processed and how you can utilize the information contained within `result.boxes`.


In [ ]:
from collections import Counter

# Initialize the video
cap = cv2.VideoCapture(traffic_url_1)
truck_counts = ...  # Counter for truck appearances

# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Detect objects in the current frame
        results = model.track(frame, persist=True, verbose=False)
        result = results[0]
        
        # TODO: Complete the code to count truck appearances
        ...
        truck_counts += ...

        # Display annotated frame
        annotated_frame = result.plot()
        show_frame(annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'): # Press 'q' to exit
            break
    else:
        break

# Cleanup and display result
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(f"Total truck appearances: {truck_counts}")

## Question 2: Counting Unique Trucks in the Video

Modify the code to count the number of unique trucks that appear throughout the video. This involves tracking individual trucks across frames. Each detected truck has a unique tracking ID when tracking is enabled in YOLOv8. Utilize these IDs to identify unique trucks over the entire video. Implement the modifications necessary to count these unique instances.



In [ ]:
from collections import Counter

# Initialize the video
cap = cv2.VideoCapture(traffic_url_1)
unique_truck_ids = ...  # store unique truck IDs

# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Detect and track objects in the current frame
        results = model.track(frame, persist=True, verbose=False)
        result = results[0]

        # TODO: Modify the code to track unique truck IDs
        for i, cls_id in enumerate(result.boxes.cls.tolist()):
            if cls_id == ...:  # Check if the class is a truck
                # id: result.boxes.id[i].item()
                ...

        # Display annotated frame
        annotated_frame = result.plot()
        show_frame(annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'): # Press 'q' to exit
            break
    else:
        break

# Cleanup and display result
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(f"Number of unique trucks: {len(unique_truck_ids)}")

## Counting Objects within a Region of Interest

In this section, we explore how to focus our object detection efforts on specific areas of an image, known as Regions of Interest (ROI). Defining an ROI allows us to concentrate on particular parts of an image while ignoring irrelevant areas, making our object detection tasks more efficient and relevant.

We will utilize a custom class, `ObjectCounter`, designed to count unique objects within the specified ROI. The ROI is defined by two points: the top-left corner `(x1, y1)` and the bottom-right corner `(x2, y2)`. These points outline a rectangular area in the image where our object detection and counting will be focused.

In [ ]:
class ObjectCounter:
    """
    A class for counting unique objects of specific classes within a specified region of interest (ROI) in video frames.
    Now processes detection results directly.
    

    Attributes:
        roi (tuple): The region of interest, defined as (x1, y1, x2, y2).
        class_names (list): A list of class names to be counted.
        unique_objects (dict): A dictionary to keep track of unique object IDs within the ROI, categorized by class.

    Methods:
        is_within_roi(box): Determines whether an object's bounding box is within the ROI.
        update(result): Processes and updates counts based on the detection result.
        get_count(): Returns the count of unique objects detected within the ROI, categorized by class.
    """

    def __init__(self, roi, class_names=None):
        """
        Initializes the ObjectCounter with a region of interest.

        Args:
            roi (tuple): The region of interest, defined as (x1, y1, x2, y2).
            class_names (list or None): Specific class names to count within the ROI.
        """
        self.roi = roi
        self.unique_objects = {}
        self.class_names = class_names

    def is_within_roi(self, box):
        """
        Determines whether the center of an object's bounding box is within the region of interest.
        
        - (x1, y1) are the coordinates of the top-left corner of a region within the image. Here, x1 corresponds to the column index, and y1 corresponds to the row index.
        - (x2, y2) are the coordinates of the bottom-right corner of the region. Similarly, x2 corresponds to the column index, and y2 corresponds to the row index.

        Args:
            box (list or numpy.ndarray): The bounding box of an object, in the format [x1, y1, x2, y2].

        Returns:
            bool: True if the object is within the ROI, False otherwise.
        """
        x_center = (box[0] + box[2]) / 2
        y_center = (box[1] + box[3]) / 2
        return self.roi[0] < x_center < self.roi[2] and self.roi[1] < y_center < self.roi[3]

    def update(self, result):
        """
        Processes the detection result and updates the counts of unique objects within the ROI.

        Args:
            result (ultralytics.YOLOv5.detect.Detections): The detection results from YOLOv5.
        """
        boxes = [box.xyxy[0].cpu().numpy().flatten() for box in result.boxes]
        ids = [box.id.item() for box in result.boxes]
        classes = [result.names[box.cls.item()] for box in result.boxes]

        for box, obj_id, cls_name in zip(boxes, ids, classes):
            if self.is_within_roi(box):
                # Ensure we're tracking this class, and update the counter if so
                if self.class_names is None or cls_name in self.class_names:
                    if cls_name not in self.unique_objects:
                        self.unique_objects[cls_name] = set()
                    self.unique_objects[cls_name].add(obj_id)

    def get_count(self):
        """
        Returns the count of unique objects detected within the ROI, categorized by class.

        Returns:
            dict: The counts of unique objects within the ROI, categorized by class.
        """
        return {cls: len(ids) for cls, ids in self.unique_objects.items()}

In [ ]:
import cv2  # Make sure OpenCV is installed and imported

def show_frame(img, roi=None):
    """
    Displays an image frame with an optional ROI. Automatically detects the execution environment.
    
    Args:
        img (numpy.ndarray): The image frame to be displayed.
        roi (tuple, optional): The region of interest defined as (x1, y1, x2, y2). If None, no ROI is drawn.
    """
    # Draw the ROI on the frame if specified
    if roi is not None:
        x1, y1, x2, y2 = roi
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)  # Draw rectangle with green line

    try:
        from google.colab.patches import cv2_imshow
        from IPython.display import clear_output
        clear_output(wait=True)  # Clear the output to prevent image accumulation
        cv2_imshow(img)  
    except ImportError:
        cv2.imshow("Frame with ROI", img)

In [ ]:
# Define the region of interest and classes of interest
roi = (100, 100, 400, 400)  # Adjust based on your video content
class_names = ['truck', 'car', 'bicycle']  # Count these classes. Set to None to count all classes.

object_counter = ObjectCounter(roi, class_names)

cap = cv2.VideoCapture(traffic_url_1)

# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Detect and track objects in the current frame
        results = model.track(frame, persist=True, verbose=False)
        result = results[0]

        # Update the object counter with detected objects
        object_counter.update(result)

        # Display annotated frame (this part remains unchanged)
        annotated_frame = result.plot()
        show_frame(annotated_frame, roi)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
            break
    else:
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

In [ ]:
print("Unique objects within ROI:", object_counter.get_count())

## Question 4: Calculating the Lane-wise Difference in Truck Counts

In this exercise, you will modify the object tracking system to differentiate between trucks passing through two distinct lanes of a road: the left lane and the right lane. You will define two Regions of Interest (ROIs), one for each lane, and count the number of unique trucks that pass through each. Finally, you will calculate the difference between the number of trucks passing through the right lane and the left lane.

### Instructions:

1. Define two separate ROIs: one for the left lane and one for the right lane. These should be based on your video's specific lane positions.
2. Use two instances of the `ObjectCounter` class, one for each lane, to count the number of trucks passing through.
3. After processing the video, calculate the difference between the counts of unique trucks in the right lane and the left lane.

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(8, 4))
plt.imshow(annotated_frame[:,:,::-1])  # Display the image in RGB format
plt.show()  # Display the plot

In [ ]:
# Define the ROIs for the left and right lanes
left_lane_roi = ... # (x1_left, y1_left, x2_left, y2_left)
right_lane_roi = ... #(x1_right, y1_right, x2_right, y2_right)

# Initialize the ObjectCounter for each lane
left_lane_counter = ...
right_lane_counter = ObjectCounter(..., class_names=[...])

# Process video frames
cap = cv2.VideoCapture(traffic_url_1)  # Replace 'traffic_url_1' with your video file path
while cap.isOpened():
    success, frame = cap.read()
    if success:
        
        # Detect and track objects in the current frame
        ...

        # Display annotated frame with the ROIs
        annotated_frame = result.plot()
        show_frame(annotated_frame, left_lane_roi)
        show_frame(annotated_frame, right_lane_roi)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
            break
    else:
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Calculate and print the difference in truck counts between the lanes
left_lane_trucks = left_lane_counter.get_count().get('truck', 0)
right_lane_trucks = right_lane_counter.get_count().get('truck', 0)
difference = right_lane_trucks - left_lane_trucks
print(f"Truck count difference (Right lane - Left lane): {difference}")

In [ ]:
left_lane_trucks, right_lane_trucks

## Extra: [Workouts Monitoring using Ultralytics](https://docs.ultralytics.com/guides/workouts-monitoring/)

In [ ]:
from ultralytics import YOLO
from ultralytics.solutions import ai_gym
import cv2

model = YOLO("yolov8n-pose.pt")
cap = cv2.VideoCapture("path/to/video")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter("workouts.avi",
                                cv2.VideoWriter_fourcc(*'mp4v'),
                                fps,
                                (w, h))

gym_object = ai_gym.AIGym()  # init AI GYM module
gym_object.set_args(line_thickness=2,
                    view_img=True,
                    pose_type="pushup",
                    kpts_to_check=[6, 8, 10])

frame_count = 0
while cap.isOpened():
    success, frame = cap.read()
    if not success:
      print("Video frame is empty or video processing has been successfully completed.")
      break
    frame_count += 1
    results = model.predict(frame, verbose=False)
    frame = gym_object.start_counting(frame, results, frame_count)
    cv2.imshow("Push-up Detection", frame)
    video_writer.write(frame)

cv2.destroyAllWindows()
video_writer.release()